In [40]:
import pandas as pd
import numpy as np

EMA = pd.read_csv("EMA Study Data/EMA Data/Grace both Time and Event based pilr_data_export.csv")
# pd.set_option("display.max_rows", 500)

tmp = np.array( (EMA["event_type"]) )

## Indices (row number in dataframe) list which only contains indices of the time survey "started ".
starts = np.where(np.isin(tmp, ["survey_started"]))
starts = starts[0][1:]

## Indices (row number in dataframe) list which only contains indices of the time survey was "submitted".
ends = np.where(np.isin(tmp, ["survey_submitted"]))
ends = ends[0][1:]

## *Alert*

### Why so many for loops are there?

**1) Location of "Yes" and "No" was inconsistent - usually next to the start of survey but two or more blocks further now and then,,**   
**2) Location of responses was also inconsistent as mentioned above.**   
   
That's why I was supposed to use AT LEAST two for loops.

In [41]:
import csv

## Dictionary for response types
res = {"0" : "Playing",
       "1" : "Talking",
       "2" : "Petting",
       "3" : "TV / Radio",
       "4" : "Eating / Cooking",
       "5" : "Moved It"}

## Row number in will-be-created file
rowid = 1

## Write a csv file
with open('EMA Study Data/Test Data/(Jun)0224.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["RowID", "Modality", "Proximity", "Start Window", "End Window"])
    
    ## How many "starts"?
    for i in range(len(starts)):
        
        ## Intervel between starts[i] and ends[i]
        for j in range(starts[i], ends[i]):
            if EMA.loc[j]["response"] == "Yes":  # Check where is"Yes" throughout the interval .
                
                ## Narrowed interval between "Yes" and ends[i]
                for idx in range(j, ends[i]):
                    if type(EMA.loc[idx]["responses"]) is str:  # "Responses" column consists of Nonetype(NaN) & string(informative).
                        
                        ## Write the information we want to extract in the new csv file
                        for value in EMA.loc[idx]["response_values"][1:len(EMA.loc[idx]["response_values"])-1].split(','):
                            start_time = EMA["metadata: timestamp (local)"].loc[j] # start(& end)_time's form : string -> pd.datetime
                            end_time = pd.to_datetime(start_time, infer_datetime_format=True) + pd.Timedelta(pd.offsets.Minute(15))
                            
                            writer.writerow([rowid, res[ value[2] ], "proximity", start_time, end_time]) # Write data one (row) by one..
                            rowid += 1